<a href="https://colab.research.google.com/github/mengyaoo/FNL_GenesSelection/blob/main/07_TC1_Feature_Selection_For_Each_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the libraries

In [2]:
from __future__ import print_function

import os, sys, gzip, glob, json, time, argparse
import pandas as pd
import numpy as np
import keras
from pandas.io.json import json_normalize

from pandas.io.json import json_normalize
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.utils import to_categorical
from keras import backend as K
from keras.layers import Input, Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten
from keras import optimizers
from keras.optimizers import SGD, Adam, RMSprop
from keras.models import Sequential, Model, model_from_json, model_from_yaml
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau

from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import sklearn.manifold as sk_manif
from keras.utils import to_categorical
from google.colab import drive
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline

# Data Preparation   

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Read features and output files 
TC1data15 = pd.read_csv("/content/drive/My Drive/FNL_TC1/TC1-S1-data15-genename.tsv", sep="\t", low_memory = False)

In [11]:
# dftm_FPKM_UQ = pd.read_csv('/content/drive/My Drive/FNL_TC1/dftm_FPKM_UQ.tsv', sep='\t', low_memory=False)

In [5]:
outcome = pd.read_csv('/content/drive/My Drive/FNL_TC1/TC1-outcome-data15-projectname.tsv', sep='\t')

In [9]:
def encode(data): 
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

In [ ]:
# outcome = encode(outcome['Project_id'])

In [17]:
TC1data15['project_id_name'] = outcome['Project_id_name']
TC1data15['project_id'] = outcome['Project_id']


In [18]:
TC1data15

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,NIPAL3,LAS1L,ENPP4,SEMA3F,CFTR,ANKIB1,CYP51A1,KRIT1,RAD52,MYH16,BAD,LAP3,CD99,HS3ST1,AOC1,WNT16,HECW1,MAD1L1,LASP1,SNX11,TMEM176A,M6PR,KLHL13,CYP26B1,ICA1,DBNDD1,ALS2,CASP10,CFLAR,...,WASIR1.1,IL3RA.1,SHOX.1,ASMT.1,AKAP17A.1,CSF2RA.1,CRLF2.1,ZBED1.1,RNA5SP498.1,TRPC6P.1,RP13-297E16.4.1,DHRSX-IT1.1,CD99P1.1,RPL14P5.1,LINC00685.1,DDX11L16.1,TCEB1P24.1,LL0YNC03-29C1.1.1,KRT18P53.1,LINC00102.1,RP13-297E16.5.1,FABP5P13.1,ASMTL-AS1.1,LINC00106.1,DPH3P2.1,RP11-309M23.1.1,AMDP1.1,BX649553.1.1,BX649553.2.1,BX649553.3.1,BX649553.4.1,MIR3690.1,AJ271736.10.1,Metazoa_SRP.305,AJ271736.1.1,MIR6089.1,RP13-465B17.5.1,RP13-465B17.4.1,project_id,project_id_name
0,1.829525,0.000000,2.062846,0.913543,0.809255,0.365063,1.315347,1.936558,0.657487,1.437908,0.551174,0.719120,1.393872,1.118917,1.987483,0.000000,1.393437,0.257243,1.161696,1.141140,0.0,1.644513,1.765476,2.090856,0.549029,0.718534,0.0,0.0,0.750320,1.912752,1.141035,1.480509,1.567273,0.164270,0.101726,1.313405,1.158674,1.141063,0.798311,1.360485,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,TCGA-OV
1,1.541309,0.000000,1.815986,0.593689,0.557331,0.608742,1.376983,1.592561,0.569209,1.103958,0.478385,1.033488,1.008026,0.758012,1.681421,1.453353,1.018066,0.223281,0.788680,0.693349,0.0,2.098755,1.643610,2.391891,0.884490,0.000000,0.0,0.0,1.128794,1.836294,0.778508,1.743826,1.415378,0.000000,0.173718,0.749825,1.539502,0.676204,0.761320,0.964733,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,TCGA-OV
2,1.282789,0.000000,1.895928,0.571531,0.332523,0.576545,1.171834,1.351888,0.836259,1.423673,0.200690,0.874702,0.876875,0.904634,1.109651,0.617534,0.904555,0.000000,0.724887,0.834916,0.0,1.567345,1.350662,2.029576,0.246144,2.193634,0.0,0.0,0.820252,1.581194,0.813868,1.547321,1.086540,0.000000,0.000000,0.464639,1.080810,0.456927,0.652686,0.725820,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,TCGA-OV
3,1.388260,0.000000,1.905475,0.399374,0.259750,0.219200,0.494936,1.822393,0.619865,0.793788,0.221321,0.567275,1.334915,0.984844,1.915951,0.458862,0.611663,0.000000,0.341496,0.735198,0.0,1.955291,1.635719,2.151499,0.000000,1.956215,0.0,0.0,0.867816,1.938688,1.049283,2.416171,1.523516,0.000000,0.000000,0.663285,0.998184,0.468721,0.456932,0.606861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,TCGA-OV
4,1.389160,0.530588,2.316735,0.349455,0.189075,0.623428,0.801534,1.685248,0.784901,0.922292,0.489743,0.968513,1.126255,0.844110,0.942073,0.000000,0.995927,0.169634,0.783794,0.614325,0.0,2.429317,1.840638,2.187733,0.238939,1.196076,0.0,0.0,1.040287,1.824699,0.927491,1.818541,1.317428,0.000000,0.000000,1.098857,1.070776,0.503460,0.218645,0.819567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,TCGA-OV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,1.897300,0.000000,1.829995,0.656801,0.001333,0.528604,2.618544,1.993496,1.487510,1.049062,0.000000,0.258848,1.455348,0.853410,1.215961,0.000000,0.571925,0.644190,0.536399,0.215012,0.0,1.704930,2.033992,2.059344,0.000000,0.000000,0.0,0.0,0.811806,1.861269,1.084387,2.724172,1.159409,0.000000,0.000000,0.193960,1.282674,0.685311,0.463778,0.899169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [86]:
def gene_cancer(cancer_name,top_gene_num=10):
  np.random.seed(123)
  cancer = TC1data15[TC1data15['project_id_name'] == cancer_name]
  non_cancer = TC1data15[TC1data15['project_id_name'] != cancer_name].sample(n = 300) 
  data_cancer = cancer.append(non_cancer)
  # define the model
  model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
  # fit the model
  model.fit(data_cancer.iloc[:,:-2], data_cancer['project_id'])
  importance = model.feature_importances_
  feature = pd.DataFrame()
  feature['name'] = TC1data15.columns[:-2]
  feature['importance'] = importance
  feature_sort = feature.sort_values(by='importance',ascending=False)
  feature_sort_selected = feature_sort.iloc[:top_gene_num,:]
  return feature_sort_selected

# TCGA-PRAD

In [89]:
gene_cancer('TCGA-PRAD',10)

,name,importance
21705,TRGC1,0.007967
10081,STEAP2,0.007273
17106,CPNE4,0.007123
44755,ZNF350,0.006913
2780,NME4,0.006663
44881,AP000438.2,0.006642
43339,FLJ20021,0.005266
8434,CREB3L4,0.004556
15659,TMPRSS2,0.004296
5625,ABCC4,0.004140


# TCGA-THCA

In [90]:
gene_cancer('TCGA-THCA',10)

,name,importance
34234,PTCSC2,0.007389
14160,TCERG1L,0.006344
15178,IDH2,0.005861
16808,SLC4A5,0.005768
8954,IGSF1,0.005677
4444,ST3GAL5,0.005602
6841,CNTN6,0.005377
8871,CDCA5,0.005374
47770,RP11-274H2.5,0.005129
34251,RP11-231P20.2,0.004957


# TCGA-OV

In [91]:
gene_cancer('TCGA-OV',10)

,name,importance
3232,NHP2P1,0.007160
57644,FP671120.6,0.006746
32979,RP11-761N21.2,0.006481
51623,RP11-380M21.2,0.006204
58500,RP11-2K6.1,0.005822
50900,MIR4478,0.005403
35386,BANF1P3,0.005339
5294,RAMP3,0.004932
30612,RPS18P12,0.004900
55243,RP11-53M11.5,0.004615


# TCGA-LUAD

In [92]:
gene_cancer('TCGA-LUAD',10)

,name,importance
17076,SFTA2,0.005912
15981,SFTPA2,0.005864
5428,C4BPA,0.005215
9208,GGTLC1,0.004984
3204,PON3,0.004218
12465,SFTPB,0.003967
633,ROS1,0.003934
12898,KCNS3,0.003530
6445,NAPSA,0.003356
34381,RP11-59D5__B.2,0.003168


# TCGA-COAD

In [93]:
gene_cancer('TCGA-COAD',10)

,name,importance
3935,APOBEC1,0.006500
195,NOX1,0.005622
20012,TRIM31,0.004784
12036,LINC00483,0.004769
9748,GAL3ST2,0.004675
1225,LLGL2,0.004671
1893,CERS4,0.004493
51764,AC024592.9,0.004070
8877,MDH2,0.003838
6737,EPHB2,0.003796


# TCGA-LIHC

In [94]:
gene_cancer('TCGA-LIHC',10)

,name,importance
1634,APOB,0.007513
1799,F11,0.006735
2136,SERPIND1,0.006633
23156,APOC1P1,0.006384
95,PON1,0.006293
252,CD9,0.004821
11339,KLKB1,0.004485
8355,F13B,0.004134
4237,KNG1,0.004122
4187,AGXT2,0.004087


# TCGA-BRCA

In [95]:
gene_cancer('TCGA-BRCA',10)

,name,importance
2925,TRPS1,0.007008
7391,LMX1B,0.004123
9399,THRSP,0.003993
49229,RP11-401O9.3,0.003846
26777,CT62,0.003812
26416,RP11-123K19.1,0.003680
25694,RP11-428L9.2,0.003479
4013,SAYSD1,0.003422
5771,ID1,0.003411
11540,TMEM246,0.002963


# TCGA-STAD

In [96]:
gene_cancer('TCGA-STAD',10)

,name,importance
52515,RP11-3P17.5,0.007498
23916,RP3-340B19.2,0.006261
37411,CTD-2184D3.1,0.006177
54958,RP11-174N3.4,0.005531
48632,RP11-750B16.1,0.005288
57365,FP236383.12,0.004759
14841,RP1-278E11.3,0.004595
5790,UROD,0.004464
37171,AL161626.1,0.004285
5819,PRDX5,0.004117


# TCGA-UCEC

In [97]:
gene_cancer('TCGA-UCEC',10)

,name,importance
9987,FGF18,0.004596
4865,ELL2,0.003926
38508,CEBPA,0.003719
12020,PBX3,0.003673
1904,NAT14,0.003224
56408,RP11-50I19.2,0.003159
6407,ZNF428,0.003123
16072,PRAME,0.002918
16716,SBK1,0.002903
17999,CCDC167,0.002827


# TCGA-KIRC

In [98]:
gene_cancer('TCGA-KIRC',10)

,name,importance
28518,RP11-807H17.1,0.006784
7744,ENPEP,0.006190
22905,LINC00887,0.005933
29214,LINC01428,0.005694
20842,RNU6-339P,0.005321
15187,TMEM30B,0.005012
16216,SAPCD2,0.005009
14753,CDH5,0.004948
23034,SEC14L6,0.004738
10564,RECQL4,0.004553


# TCGA-LGG

In [99]:
gene_cancer('TCGA-LGG',10)

,name,importance
15186,FAM181B,0.007361
10876,RFTN2,0.007354
2875,SCG3,0.007234
3139,CACNG7,0.007083
28317,AC018647.3,0.004154
2121,HCN2,0.004032
16245,NAP1L3,0.004006
9882,GRIA1,0.003987
12837,DNAJC18,0.003961
135,BAIAP2L1,0.003910
